In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from library.Constants import CURRENCY_PAIRS
from library.Indicators import SMA, BB, RSI
import library.MetaTrader_Data as mtd

MetaTrader 5 initialized successfully.


In [2]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 500
SMA_PERIODS = [14,21, 50]
RSI_OVERSOLD = 20
RSI_OVERBOUGHT = 80
TRIGGER_SMA_PERIOD = 50

In [3]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Fetched data for pair: USDJPY
Fetched data for pair: EURUSD
Fetched data for pair: GBPUSD
Fetched data for pair: EURGBP
Fetched data for pair: CADCHF
Fetched data for pair: EURJPY
Fetched data for pair: AUDUSD
Fetched data for pair: USDCNH
Fetched data for pair: EURCHF
Fetched data for pair: NZDUSD


In [4]:
for symbol, df in mtd.data.items():
    for period in SMA_PERIODS:
        SMA(df, period)
        BB(df, period, std=2)
        RSI(df, period)

In [6]:
for symbol, df in mtd.data.items():
    # Create lists to store the crossover points
    cross_from_below = []  # To store green dots (close crossing SMA from below)
    cross_from_above = []  # To store red dots (close crossing SMA from above)
    
    # Loop through the dataframe row by row to detect crossings
    for index in range(1, len(df)):
        prev_close_price = df.iloc[index - 1]['close']
        prev_sma = df.iloc[index - 1][f'sma-{TRIGGER_SMA_PERIOD}']
        close_price = df.iloc[index]['close']
        sma = df.iloc[index][f'sma-{TRIGGER_SMA_PERIOD}']

        # Detect crossings from below
        if prev_close_price < prev_sma and close_price > sma:
            cross_from_below.append((df.iloc[index]['time'], close_price))
        
        # Detect crossings from above
        elif prev_close_price > prev_sma and close_price < sma:
            cross_from_above.append((df.iloc[index]['time'], close_price))
    
    # Plotting the lines for close price and SMA
    fig = go.Figure()

    # Add the close price line
    fig.add_trace(
        go.Scatter(x=df['time'], y=df['close'], mode='lines', name='Close Price')
    )
    
    # Add the SMA line
    fig.add_trace(
        go.Scatter(x=df['time'], y=df[f'sma-{TRIGGER_SMA_PERIOD}'], mode='lines', name='SMA', line_color='blue')
    )
    
    # Add green dots where the close price crosses the SMA from below
    if cross_from_below:
        fig.add_trace(
            go.Scatter(
                x=[x[0] for x in cross_from_below], 
                y=[x[1] for x in cross_from_below],
                mode='markers', 
                marker=dict(color='green', size=6), 
                name='Cross from Below'
            )
        )
    
    # Add red dots where the close price crosses the SMA from above
    if cross_from_above:
        fig.add_trace(
            go.Scatter(
                x=[x[0] for x in cross_from_above], 
                y=[x[1] for x in cross_from_above],
                mode='markers', 
                marker=dict(color='red', size=6), 
                name='Cross from Above'
            )
        )
    
    # Update layout
    fig.update_layout(
        title=f'{symbol}: Close Price and SMA with Crossings',
        template="plotly_dark",
        xaxis_title="Time",
        yaxis_title="Price",
        height=600
    )
    
    fig.show()
